In [ ]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

from plotly import express as px
from typing import List, Tuple
import pickle
import numpy as np

In [ ]:
import sys
sys.path.append('../../../../')

## load

In [ ]:
with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/03_04_with_pesto_and_coord/folds_training_dicts.pkl', 'rb') as f:
    folds_training_dicts = pickle.load(f)

In [ ]:
fold = folds_training_dicts[0]
fold.keys()

In [ ]:
input_data = fold['components_train']
coordinates = fold['coordinates_train']
size_value = fold['sizes_train']
n_patches_hot_encoded_value = fold['num_patches_train']
max_number_of_patches = 10

## debug pairwise transition

In [ ]:
from model import create_masked_inputs

features, pairwise_distances = create_masked_inputs(
        input_data, coordinates, size_value, n_patches_hot_encoded_value, max_number_of_patches)

In [ ]:
# px.imshow(pairwise_distances[0][:, :, 0])

In [ ]:
# px.imshow(pairwise_distances._keras_mask[0])

In [ ]:
from models.patch_to_score.models.modules.scannet_layers import GaussianKernel, MaskedDense, initialize_GaussianKernel, initialize_GaussianKernelRandom

In [ ]:
# flat_distances = pairwise_distances.numpy().flatten()
# flat_distances = flat_distances[flat_distances != 0]
# px.histogram(flat_distances[::10])

In [ ]:
N = 20
num_heads = 1
# initial_values = initialize_GaussianKernelRandom([[0, 200]], N, 'diag')
initial_values = [np.array([np.linspace(0, 200, N)]), np.array([[(200 / (N/ 4)) for _ in range(N)]])]
kernel = GaussianKernel(N, initial_values, 'diag')

layer_norm = tf.keras.layers.LayerNormalization()
dense = MaskedDense(num_heads, use_bias=False)

after_kernel = kernel(pairwise_distances)
after_layer_norm = layer_norm(after_kernel)
after_dense = dense(after_layer_norm)

In [ ]:
x = pairwise_distances.numpy().flatten()
ys = [after_kernel[:, :, :, i].numpy().flatten() for i in range(N)]
px.scatter(x=x[::1111], y=[y[::1111] for y in ys])

In [ ]:
# px.imshow(pairwise_distances[0][:, :, 0])

In [ ]:
px.imshow(after_dense[0][:, :, 0])